Membangun model klasifikasi emosi wajah berbasis deep learning dengan menggunakan dataset yang telah dikelompokkan ke dalam folder untuk masing-masing emosi. Dataset diolah menggunakan augmentasi data untuk memperluas variasi data pelatihan, sehingga model menjadi lebih robust.

Dataset dan Emosi: Dataset citra wajah dipisahkan ke dalam folder train (untuk pelatihan) dan test (untuk pengujian). Kategori emosi yang akan diklasifikasikan didefinisikan dalam daftar emotions, misalnya 'angry', 'neutral', dll.

Augmentasi Data: Augmentasi diterapkan pada data pelatihan dengan menggunakan ImageDataGenerator untuk melakukan operasi seperti rescaling (normalisasi piksel), shear, zoom, dan flipping horizontal. Data pengujian hanya dinormalisasi tanpa augmentasi.

Generator Data: flow_from_directory digunakan untuk secara otomatis membaca dataset dari folder, mengubah ukuran gambar menjadi 48x48 piksel (sesuai dengan dimensi model), dan mengubahnya menjadi batch yang siap diproses oleh model.

Fungsi: Proses ini memastikan bahwa model CNN akan menerima data dalam format yang sesuai, dengan augmentasi yang membantu meningkatkan performa model pada data baru, sementara data pengujian digunakan untuk mengevaluasi akurasi model pada data yang tidak dilatih.

In [2]:
# Import necessary libraries
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load the dataset
train_dir = './train'
test_dir = './test'

# Define the emotions
emotions = ['angry','neutral', 'disgust', 'fear', 'happy','sad', 'surprise']

# Define the image dimensions
img_height, img_width = 48, 48

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 57544 images belonging to 7 classes.
Found 14244 images belonging to 7 classes.


**Membangun Model CNN**

Pelatihan model Convolutional Neural Network (CNN) untuk klasifikasi emosi wajah berdasarkan gambar. Model ini menggunakan TensorFlow dan Keras, memanfaatkan GPU untuk mempercepat pelatihan. Setelah mendeteksi GPU yang tersedia, model dibangun dengan beberapa lapisan convolutional untuk mengekstraksi fitur gambar, diikuti dengan lapisan pooling untuk mereduksi dimensi data. Setelah fitur diekstraksi, data diratakan dan diteruskan ke lapisan fully connected (Dense) untuk membuat prediksi akhir. Fungsi aktivasi relu digunakan untuk non-linearitas pada lapisan convolutional, sementara softmax digunakan pada lapisan output untuk menghasilkan distribusi probabilitas atas 7 kelas emosi. Model dikompilasi dengan optimizer Adam, loss function categorical_crossentropy, dan metrik akurasi. Pelatihan dilakukan selama 70 epoch menggunakan data training yang dihasilkan dari generator dengan augmentasi gambar, sedangkan data validasi digunakan untuk memantau performa selama pelatihan. Akhirnya, model dievaluasi pada data testing untuk mendapatkan akurasi akhir.








### Build the model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.config.list_physical_devices('GPU'))

# Set the GPU as the default device
with tf.device('/GPU:0'):
    # Build the model
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(len(emotions), activation='softmax'))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(train_generator,
                        epochs=70,
                        validation_data=test_generator)

    # Evaluate the model
    loss, accuracy = model.evaluate(test_generator)
    print(f'Test accuracy: {accuracy:.2f}')

In [4]:
model.save('emotion_detection_model_CNN.h5')

**Sequence-to-Sequence (Seq2Seq)** dengan encoder-decoder menggunakan TensorFlow dan Keras, yang dirancang untuk menangani data gambar dengan prediksi berbasis sekuensial, misalnya untuk mendeteksi emosi wajah dalam bentuk distribusi probabilitas pada 7 kelas emosi.


Dibangun dengan arsitektur Convolutional Neural Network (CNN).
Tugasnya adalah mengekstraksi fitur dari gambar input (dengan dimensi 48x48x3).
CNN menggunakan dua lapisan Conv2D diikuti oleh MaxPooling2D untuk reduksi dimensi.
Akhirnya, fitur diratakan (Flatten) dan dilanjutkan dengan lapisan dense untuk menghasilkan representasi fitur latar belakang.
Decoder:

Dibangun dengan Long Short-Term Memory (LSTM) untuk menghasilkan prediksi sekuensial berbasis keluaran dari encoder.
Fitur yang diekstrak oleh encoder direntangkan menjadi sekuens (RepeatVector(seq_len)).
LSTM pertama memproses seluruh sekuens, menghasilkan urutan fitur, dan LSTM kedua menyederhanakan representasi sekuensial ini.
Lapisan Dense dengan fungsi aktivasi softmax digunakan untuk memprediksi probabilitas dari 7 emosi pada setiap langkah keluaran.
Model Seq2Seq:

Model ini dibangun sebagai kelas kustom menggunakan Keras, menyatukan encoder dan decoder.
Fungsi call mendefinisikan alur data: input diproses oleh encoder, lalu diteruskan ke decoder.
Penggunaan GPU:

Bagian kode memeriksa GPU yang tersedia dan mengaktifkan alokasi memori dinamis untuk menghindari kesalahan alokasi memori tetap.
Pelatihan:

Model dilatih menggunakan train_generator dan test_generator, dengan augmentasi data pada gambar input.
Optimizer Adam digunakan untuk mempercepat konvergensi.
Loss function categorical_crossentropy diterapkan karena targetnya adalah klasifikasi multi-kelas.
Evaluasi:

Model diuji pada data validasi (test_generator) setelah pelatihan.
Akurasi akhir dicetak untuk mengevaluasi performa model pada data uji.
Manfaat Arsitektur:
Encoder-decoder memungkinkan model untuk menangani hubungan kompleks dalam data.
Kombinasi CNN dan LSTM sangat cocok untuk masalah yang membutuhkan pemrosesan data spasial dan sekuensial.

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Constants
img_height, img_width = 48, 48
num_emotions = 7
seq_len = 10

def build_encoder():
    model = keras.Sequential([
        layers.Input(shape=(img_height, img_width, 3)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu')
    ])
    return model

def build_decoder(encoder_output_shape):
    model = keras.Sequential([
        layers.Input(shape=(encoder_output_shape,)),
        layers.RepeatVector(seq_len),
        layers.LSTM(64, return_sequences=True),
        layers.LSTM(32),
        layers.Dense(num_emotions, activation='softmax')
    ])
    return model

class Seq2Seq(keras.Model):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        # Encode input
        encoded = self.encoder(inputs)
        
        # Decode encoded features
        decoded = self.decoder(encoded)
        
        return decoded

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print(f"Using GPU: {gpus[0]}")
    except RuntimeError as e:
        print(f"GPU Configuration Error: {e}")

# Create Model
encoder = build_encoder()
decoder = build_decoder(encoder.output_shape[1])
seq2seq = Seq2Seq(encoder, decoder)

# Compile Model
seq2seq.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

seq2seq.fit(train_generator, epochs=70, validation_data=test_generator)
loss, accuracy = seq2seq.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

Epoch 1/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 72s 38ms/step - accuracy: 0.2982 - loss: 1.7140 - val_accuracy: 0.4363 - val_loss: 1.4382
Epoch 2/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 73s 41ms/step - accuracy: 0.4425 - loss: 1.4358 - val_accuracy: 0.5010 - val_loss: 1.2965
Epoch 3/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 72s 40ms/step - accuracy: 0.4905 - loss: 1.3241 - val_accuracy: 0.5224 - val_loss: 1.2599
Epoch 4/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 73s 40ms/step - accuracy: 0.5166 - loss: 1.2645 - val_accuracy: 0.5356 - val_loss: 1.2030
Epoch 5/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 75s 42ms/step - accuracy: 0.5320 - loss: 1.2184 - val_accuracy: 0.5503 - val_loss: 1.1796
Epoch 6/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 71s 40ms/step - accuracy: 0.5509 - loss: 1.1804 - val_accuracy: 0.5595 - val_loss: 1.1598
Epoch 7/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 77s 43ms/step - accuracy: 0.5642 - loss: 1.1477 - val_accuracy: 0.5603 - val_loss: 1.1660
Epoch 8/70
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 74s 41ms/step - accuracy: 0.5811 -

In [6]:
model.save('emotion_detection_model_seq2seq.h5')

**Program untuk mendeteksi emosi wajah secara real-time menggunakan kamera dan dua model deep learning yang telah dilatih sebelumnya (Seq2Seq dan CNN).**

In [1]:
# Import necessary libraries
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the trained models
model_v1 = tf.keras.models.load_model('emotion_detection_model_seq2seq.h5')
model_v2 = tf.keras.models.load_model('emotion_detection_model_CNN.h5')

model_v1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_v2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the emotions
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral','sad','surprise']

# Define the image dimensions
img_height, img_width = 48, 48

# Open the default camera
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Check if the frame is not empty
    if not ret:
        print("Cannot receive frame")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Loop through the detected faces
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Extract the face from the frame
        face = gray[y:y+h, x:x+w]

        # Resize the face to the input shape of the model
        face = cv2.resize(face, (img_height, img_width))

        # Convert the face to a 3-channel image
        face = cv2.cvtColor(cv2.cvtColor(face, cv2.COLOR_GRAY2BGR), cv2.COLOR_BGR2RGB)

        # Expand the dimensions of the face to match the input shape of the model
        face = np.expand_dims(face, axis=0)
        face = face / 255.0

        # Make a prediction on the face using both models
        prediction_v1 = model_v1.predict(face)
        prediction_v2 = model_v2.predict(face)

        # Get the predicted emotions and their probabilities
        predicted_emotion_v1 = emotions[np.argmax(prediction_v1)]
        predicted_emotion_v2 = emotions[np.argmax(prediction_v2)]

        # Display the predicted emotions and their probabilities on the frame
        cv2.putText(frame, f"seq2seq: {predicted_emotion_v1} ({np.max(prediction_v1)*100:.2f}%)", (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 1)
        cv2.putText(frame, f"CNN: {predicted_emotion_v2} ({np.max(prediction_v2)*100:.2f}%)", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 1)

        # Display the probabilities of each emotion for both models
        for i, emotion in enumerate(emotions):
            cv2.putText(frame, f"{emotion}: {prediction_v1[0][i]*100:.2f}% (seq2seq) / {prediction_v2[0][i]*100:.2f}% (CNN)", (x, y+h+10+i*15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Exit on press 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━